In [1]:
import json
from pprint import pprint


In [98]:

import pandas as pd
import numpy as np
from datetime import datetime
from enum import Enum
from abc import ABC, abstractmethod

In [99]:
#a = open('feeddata_10.json')


In [100]:
#'KR is for Korea
#4 for derivatives
#'Then type 1 for future ,2 for call, 3 for put
#'0 no clue
#'Then 1 for maxi or 5 for mini
#'Then 2 letters expiry code, 1 one is year, second month
#'Then 3 digits for the strike
#'Then 1 for isin checksum


In [101]:
data = json.load(open('feeddata_10.json'))

In [102]:
#aPD = pd.DataFrame.from_records(data)
#books = aPD.book.drop_duplicates().str.split('01')
#books = aPD.book.drop_duplicates() #check 1 if maxi 5 if mini
#aTest = books.to_frame

In [103]:
class CONTRACTTYPE(Enum):
    UNSET = 0
    FUTURE = 1
    CALL = 2
    PUT = 3
    
#class Strike(Enum):
#    for i in range(50)+100:
#         str(i+2.5*i) = i-99

In [204]:
# class Books():
    
#     contractType = CONTRACTTYPE.UNSET
#     book = []

    
#     def __init__(self,*args):
#         self.contractType = args[0]
#         if args[1]:
#             self.strike = args[1]
        
#     def setStrike(aStrike):
#         self.strike = aStrike

#     def updateBook(aTickBook):
#         self.book = aTickBook
#         self.updateTime = ''
#         self.timeToExpiry = ''
        
#     def getVolume(level, isBid):
#         return 0
        
#     def getBestBid():
#         return 0
    
#     def getBestAsk():
#         return 0

In [206]:
class aSimpleBook():
    
    aStrikeBook = {}
    
    def getBestBid_Price(self,aStrike):
        return aStrikeBook["aStrike"]['bid_price_0']
    
    def getBestAsk_Price(self,aStrike):
        return aStrikeBook["aStrike"]['ask_price_0']
    
    def getBestBid_Volume(self,aStrike):
        return aStrikeBook["aStrike"]['bid_volume_0']
    
    def getBestAsk_Volume(self,aStrike):
        return aStrikeBook["aStrike"]['ask_volume_0']
    
    def updateBook(self,aNewTick):
        self.aStrikeBook["aStrike"] = aNewTick
        


In [192]:
class Traders(ABC):


    @abstractmethod
    def getPnL(self):
        pass
    @abstractmethod
    def getPosition(self):
        pass
    @abstractmethod
    def update(self):
        pass

In [193]:
class TraderTest(Traders):
    Position = {}
    LastFutureValue = 0
    cashPnL = 0
    PnL = 0
    theStrategies = []
    theBook = aSimpleBook
    
    def __init__(self, aBunchOfStrategies):
        theStrategies = aBunchOfStrategies
    
    def getPnL(self):
        return self.PnL
    
    def getPosition(self, strike): #options
        return self.Position['Strike']
    
    def getPosition(self):  #futures
        return self.Position['000']
    
    def updatePnL(self, aVolumeChange,aStrike):
        
        self.PnL = 0
        
        if(aVolumeChange > 0):  
            cashPnL = cashPnL - aVolumeChange*self.theBook[aStrike]['ask_price_0']  #only crossing the spread here, assuming always enough volume
        elif(aVolumeChange < 0):
            cashPnL = cashPnL - aVolumeChange*self.theBook[aStrike]['bid_price_0']
            
        for key in Position.keys:
            self.PnL = self.PnL + self.Position['key']*(self.theBook[key]['ask_price_0']+self.theBook[key]['bid_price_0'])/2
        
        self.PnL = self.PnL + cashPnL
        
        return self.PnL
    
    def update(self,aTick):
        
        self.theBook.updateBook(aTick["rawStrikeBook"])
            
        for strat in theStrategies:
            volumeChange = strat.getAction(aTick, theBook)
            self.Position[aTick["Strike"]] = self.Position[aTick["Strike"]] + volumeChange
        
        myPnL = self.updatePnL(volumeChange,aTick["Strike"])
        return myPnL
        

In [194]:
class Strategy(ABC):
    
    @abstractmethod
    def getAction(self,aTick):
        return 0

In [195]:
class StrategyTest():
    
    def getAction(aTick, LastFutureValue, aBook):
        
        myQty = 0 
        
        if(aTick["contract"] == CONTRACTTYPE.CALL):
            if aTick["midPoint"] > LastFutureValue - float(aTick["Strike"])*getDiscount(aTick) + putPrice:
                myQty = -1 
                
                
        return myQty
        
        
    
    
    

In [196]:
def isCorrectISINCHECK(aISIN):
    #TODO
    return True

In [197]:
def getDiscount(aTick):
    
    rate = 0.025
    
    return np.exp( -rate * (aTick["timeToExpiry"].days + aTick["timeToExpiry"].seconds / (60*60*24))/365)
    
    

In [198]:
import calendar
import sys
from dateutil import tz

def getExpiryDate(anExpiryCode):
    #hardcoded version for now. 
    myString_date = ''
    
    myString_year = ''
    myString_month = ''
    myString_day = ''
    
    #get year code
    if anExpiryCode[0] == 'L':
        myString_year = "2016"
    elif anExpiryCode[0] == 'M':
        myString_year = "2017"
    elif anExpiryCode[0] == 'N':
        myString_year = "2018"
    elif anExpiryCode[0] == 'P': #It looks strange, but that's KRX 
        myString_year = "2019"
    else : 
        sys.stderr.write("fatal error\n expiry year code unknown \n")
    
    #get month code
    myPotentialMonths = ['','1','2','3','4','5','6','7','8','9','A','B','C']

    if anExpiryCode[1] in myPotentialMonths:
        myString_month =  str(myPotentialMonths.index(anExpiryCode[1]))
    else : 
        sys.stderr.write("fatal error\n expiry month code unknown \n")    
    
    if myString_month == '':
        sys.stderr.write("fatal error\n expiry month code unknown \n")   
    #get day
    #get third Wednesday of every month, we will have to make bank holidays exceptions
    myString_day = str(calendar.Calendar(2).monthdatescalendar(int(myString_year), \
                                                               int(myString_month))[3][0].day)
    
                       
    myString_date = myString_year+'-'+myString_month+'-'+myString_day+' 16:00:00'

    timeZone = tz.gettz('Asia/Seoul')
            
    #print("Don't forget bank holidays exceptions")
    return datetime.strptime(myString_date, "%Y-%m-%d %H:%M:%S").replace(tzinfo=timeZone).astimezone(timeZone)


In [199]:

def utcToRightTMZ(aUTCTMZ):
    
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz('Asia/Seoul')
    
    aUTCTMZ = aUTCTMZ.replace(tzinfo=from_zone)
    localTimeStamp = aUTCTMZ.astimezone(to_zone)
    
    return localTimeStamp

In [200]:
def getBookContract(aStringNumberContract, aTypeOption):
    aContract = ''
    if aStringNumberContract == "000" and aTypeOption == "1":
        aContract = CONTRACTTYPE.FUTURE #replace with enums
    elif aTypeOption == "2":
        aContract = CONTRACTTYPE.CALL #replace with enums
    elif aTypeOption == "2":
        aContract = CONTRACTTYPE.CALL #replace with enums
    else:
        print("error, Contract Type" + str(aStringNumberContract) + " , " + str(aTypeOption) + " not recognized")
    
    return aContract

In [201]:
class simulationCrunsher():
    
    def __init__(self, strategies_):
        self.theTrader = TraderTest(strategies_)
    
    def run(self, data):
        counter = 0
        myFirstLastDone = True

        for tick in data:
            counter = counter + 1 
            myTick = dict()

            myBook = tick["book"]
            if not isCorrectISINCHECK(myBook):
                sys.stderr.write("fatal error\n ISIN CHECK IS BREAKING \n")   

            if myBook[5] == '1' and  tick['type']=='tick':   #let's forget about mini futures
                try:
                    #print(myBook)
                    myTick["contract"] = getBookContract(myBook[8:11],myBook[3])

                    timeStamp = utcToRightTMZ(datetime.utcfromtimestamp(tick["created"]/1000000))
                    #print(timeStamp)
                    myTick["created"] = timeStamp
                    myTick["received"] = utcToRightTMZ(datetime.utcfromtimestamp(tick["received"]/1000000))

                    myTempStrike = data[1]['book'][8:11]

                    if(myTick["contract"]  == CONTRACTTYPE.CALL or myTick["contract"]  == CONTRACTTYPE.PUT):
                        myTick["Strike"] = myBook[8:11]
                    #print(myBook[6:8])
                    myTick["expiry"] = getExpiryDate(myBook[6:8])

                    myTick["spread"] = (tick['ask_price_0']-tick['bid_price_0'])/2
                    myTick["midPoint"] =(tick['ask_price_0']+tick['bid_price_0'])/2

                    myTick["timeToExpiry"] = myTick["expiry"] - myTick["received"]
                    
                    myTick["rawStrikeBook"] = tick
                    
                    pnL = self.theTrader.update(myTick)
                    
                except Exception as e: 
                    print(e)
                    print(tick)
                    break

            elif myBook[5] == '1' and tick['type']=='lastdone':
                if myFirstLastDone: 
                    print("@TODO : Handle Last Done")
                myFirstLastDone = False
            elif myBook[5] == '1':
                print(tick)

            if counter > 2000 :
                break

example:

In [202]:
tick

{'ask_price_0': 3.91,
 'ask_price_1': 3.92,
 'ask_price_2': 3.93,
 'ask_price_3': 3.94,
 'ask_price_4': 3.95,
 'ask_volume_0': 3,
 'ask_volume_1': 26,
 'ask_volume_2': 8,
 'ask_volume_3': 17,
 'ask_volume_4': 44,
 'bid_price_0': 3.9,
 'bid_price_1': 3.89,
 'bid_price_2': 3.88,
 'bid_price_3': 3.87,
 'bid_price_4': 3.86,
 'bid_volume_0': 1,
 'bid_volume_1': 8,
 'bid_volume_2': 9,
 'bid_volume_3': 10,
 'bid_volume_4': 7,
 'book': 'KR4201MB3355',
 'created': 1509498004173059,
 'received': 1509498004173475,
 'type': 'tick'}

In [207]:
class runners():
    theStrats = [StrategyTest]
    
    theSimulator = simulationCrunsher(theStrats)
    
    theSimulator.run(data)
    

updateBook() missing 1 required positional argument: 'aNewTick'
{'received': 1509498000001973, 'created': 1509498000001577, 'book': 'KR4201MC3404', 'type': 'tick', 'ask_price_0': 4.15, 'ask_volume_0': 1, 'ask_price_1': 4.16, 'ask_volume_1': 6, 'ask_price_2': 4.17, 'ask_volume_2': 1, 'ask_price_3': 4.18, 'ask_volume_3': 1, 'ask_price_4': 4.2, 'ask_volume_4': 1, 'bid_price_0': 4.07, 'bid_volume_0': 1, 'bid_price_1': 4.06, 'bid_volume_1': 2, 'bid_price_2': 4.05, 'bid_volume_2': 7, 'bid_price_3': 4.04, 'bid_volume_3': 5, 'bid_price_4': 4.03, 'bid_volume_4': 6}


SIMPLE EACD MODEL : 

In [98]:

def residualTotalEACD(alpha0_, alphas_, betas_):
    

    #variables of Interest:
    errs = []
    psis = []
    xs = []

    p = len(alphas)
    q = len(betas)
    
    
    
    initialized = False
    first = True
    for idx, tick in enumerate(data):
        
        if initialized and not first :
            
            #calculate alphas and betas parts
            alphaSum = 0.0
            betaSum = 0.0            
            for idx2,alpha in enumerate(alphas_): 
                print(xs)
                alphaSum = alphaSum + alpha*xs[idx2] #doesn't matter if sum in reverse order, just means that alphas are in reverse order too
            for idx2,beta in enumerate(betas_):
                print(psis)
                if len(psis)>idx2:
                    betaSum = betaSum + beta*psis[idx2]
                
            expectedTime = alpha0_ + alphaSum +  betaSum #expected time (psi)
            psis.append(expectedTime)
            xs.append((datetime.utcfromtimestamp(tick["created"]/1000000) - myPastTime).microseconds/1000)
            

            errs.append((expectedTime - xs[p])*(expectedTime - xs[p]))

            
        elif first:
            first = False
        else:
            xs.append((datetime.utcfromtimestamp(tick["created"]/1000000) - myPastTime).microseconds/1000)
            if len(xs) == p:
                initialized = True
                
        myPastTime = datetime.utcfromtimestamp(tick["created"]/1000000)
        
        #maintain right array size
        if len(xs)>p:
            xs.pop(0)
        if len(psis)>q:
            psis.pop(0)  
            
        if idx > 2000 :
            break
    return errs

In [99]:
# if 'xs' in globals() or 'psis' in globals():
#     sys.stderr.write('xs or/and psis variables already exist')
#else:


#parameters:
alpha0 = 0.0
alphas = [1]
betas = [0]

residualTotalEACD(alpha0, alphas, betas)



[0.736]
[]
[0.096]
[0.736]
[0.057]
[0.096]
[16.239]
[0.057]
[0.214]
[16.239]
[0.085]
[0.214]
[103.117]
[0.085]
[0.048]
[103.117]
[0.05]
[0.048]
[0.057]
[0.05]
[0.044]
[0.057]
[0.045]
[0.044]
[0.051]
[0.045]
[0.042]
[0.051]
[0.052]
[0.042]
[0.043]
[0.052]
[2.975]
[0.043]
[7.883]
[2.975]
[8.83]
[7.883]
[21.142]
[8.83]
[26.078]
[21.142]
[26.531]
[26.078]
[17.872]
[26.531]
[19.086]
[17.872]
[6.225]
[19.086]
[8.251]
[6.225]
[2.319]
[8.251]
[22.996]
[2.319]
[0.132]
[22.996]
[0.057]
[0.132]
[0.056]
[0.057]
[0.969]
[0.056]
[0.227]
[0.969]
[0.202]
[0.227]
[0.074]
[0.202]
[0.436]
[0.074]
[0.092]
[0.436]
[0.045]
[0.092]
[0.044]
[0.045]
[0.049]
[0.044]
[0.045]
[0.049]
[0.051]
[0.045]
[0.046]
[0.051]
[0.043]
[0.046]
[0.05]
[0.043]
[0.043]
[0.05]
[0.044]
[0.043]
[0.767]
[0.044]
[1.061]
[0.767]
[0.787]
[1.061]
[1.149]
[0.787]
[0.436]
[1.149]
[1.043]
[0.436]
[0.088]
[1.043]
[0.393]
[0.088]
[0.317]
[0.393]
[0.426]
[0.317]
[0.636]
[0.426]
[0.054]
[0.636]
[0.073]
[0.054]
[0.071]
[0.073]
[0.082]
[0.071]
[

[0.119]
[0.0]
[0.123]
[0.949]
[0.0]
[999.244]
[0.949]
[0.121]
[999.244]
[0.0]
[0.121]
[0.881]
[0.0]
[999.257]
[0.881]
[0.093]
[999.257]
[0.078]
[0.093]
[0.149]
[0.078]
[0.075]
[0.149]
[0.069]
[0.075]
[0.087]
[0.069]
[0.142]
[0.087]
[0.072]
[0.142]
[0.131]
[0.072]
[0.07]
[0.131]
[0.068]
[0.07]
[0.079]
[0.068]
[0.321]
[0.079]
[0.0]
[0.321]
[1.03]
[0.0]
[0.0]
[1.03]
[0.133]
[0.0]
[0.086]
[0.133]
[0.186]
[0.086]
[999.914]
[0.186]
[0.169]
[999.914]
[0.096]
[0.169]
[0.079]
[0.096]
[0.188]
[0.079]
[0.104]
[0.188]
[0.57]
[0.104]
[0.0]
[0.57]
[0.24]
[0.0]
[0.0]
[0.24]
[0.345]
[0.0]
[0.0]
[0.345]
[0.113]
[0.0]
[0.374]
[0.113]
[0.0]
[0.374]
[0.113]
[0.0]
[0.203]
[0.113]
[0.08]
[0.203]
[0.344]
[0.08]
[0.0]
[0.344]
[0.135]
[0.0]
[0.569]
[0.135]
[0.0]
[0.569]
[0.131]
[0.0]
[0.341]
[0.131]
[999.872]
[0.341]
[0.0]
[999.872]
[0.222]
[0.0]
[0.323]
[0.222]
[0.0]
[0.323]
[0.145]
[0.0]
[0.329]
[0.145]
[0.0]
[0.329]
[0.128]
[0.0]
[0.096]
[0.128]
[0.081]
[0.096]
[0.086]
[0.081]
[0.458]
[0.086]
[0.0]
[0.458]


[0.4096,
 0.001521,
 261.85712400000006,
 256.8006250000001,
 0.016641,
 10615.593024000002,
 10623.218761,
 4.0000000000000074e-06,
 4.899999999999999e-05,
 0.00016900000000000012,
 1.0000000000000019e-06,
 3.599999999999998e-05,
 8.09999999999999e-05,
 9.99999999999999e-05,
 8.100000000000002e-05,
 8.596624,
 24.088463999999995,
 0.8968090000000001,
 151.585344,
 24.364096,
 0.20520899999999945,
 74.97828099999998,
 1.4737959999999968,
 165.40532099999996,
 4.1046759999999995,
 35.188624,
 427.538329,
 522.7624959999998,
 0.0056250000000000015,
 1.0000000000000019e-06,
 0.8335689999999999,
 0.5505639999999999,
 0.0006249999999999997,
 0.016384,
 0.131044,
 0.11833599999999998,
 0.002209,
 1.0000000000000019e-06,
 2.5000000000000045e-05,
 1.600000000000003e-05,
 3.599999999999998e-05,
 2.4999999999999974e-05,
 9.000000000000015e-06,
 4.9000000000000087e-05,
 4.9000000000000087e-05,
 1.0000000000000019e-06,
 0.522729,
 0.08643599999999996,
 0.07507599999999995,
 0.131044,
 0.5083690000